# Práctica 5: Generative Adversarial Network (GAN) para MNIST

En esta práctica implementamos un GAN simple para generar dígitos MNIST. Seguimos el enunciado que requiere:

## Objetivos del enunciado:
1. **Visualizar la evolución de errores**: Gráfica con la evolución de los errores del discriminador y generador durante el entrenamiento
2. **Calcular métricas del modelo**: Métricas de rendimiento del discriminador y generador

## Arquitectura GAN:
- **Generador**: Transforma ruido aleatorio en imágenes 28x28
- **Discriminador**: Clasifica imágenes como reales o generadas
- **Entrenamiento adversarial**: Ambos modelos compiten para mejorar

In [ ]:
# ============================================================================================
# SECCIÓN 1: CONFIGURACIÓN E IMPORTACIÓN DE LIBRERÍAS
# ============================================================================================

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import time
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Configuración de reproducibilidad
torch.manual_seed(42)
np.random.seed(42)

# Configuración de dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")
print(f"Versión de PyTorch: {torch.__version__}")

# Configuración del dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalizar a [-1, 1]
])

# Cargar dataset MNIST
print("\nCargando dataset MNIST...")
mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
dataloader = DataLoader(mnist_dataset, batch_size=128, shuffle=True, num_workers=2)

print(f"Dataset cargado: {len(mnist_dataset)} muestras")
print(f"Batches por época: {len(dataloader)}")

In [ ]:
# ============================================================================================
# SECCIÓN 2: DEFINICIÓN DE ARQUITECTURAS
# ============================================================================================

class Generator(nn.Module):
    """
    Generador: Convierte ruido aleatorio (z) en imágenes MNIST 28x28
    
    Arquitectura:
    - Input: Vector de ruido de dimensión z_dim (100)
    - Hidden: 2 capas densas con ReLU
    - Output: Imagen 28x28 con activación Tanh [-1, 1]
    """
    
    def __init__(self, z_dim=100, img_dim=28*28):
        super(Generator, self).__init__()
        self.z_dim = z_dim
        self.img_dim = img_dim
        
        self.net = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(True),
            nn.Dropout(0.3),
            
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Dropout(0.3),
            
            nn.Linear(512, img_dim),
            nn.Tanh()  # Output en [-1, 1]
        )

    def forward(self, z):
        """
        Forward pass del generador
        Args:
            z: Tensor de ruido (batch_size, z_dim)
        Returns:
            img: Tensor de imágenes generadas (batch_size, img_dim)
        """
        return self.net(z)


class Discriminator(nn.Module):
    """
    Discriminador: Clasifica imágenes como reales (1) o falsas (0)
    
    Arquitectura:
    - Input: Imagen aplanada 28x28 = 784
    - Hidden: 2 capas densas con LeakyReLU
    - Output: Probabilidad [0, 1] con Sigmoid
    """
    
    def __init__(self, img_dim=28*28):
        super(Discriminator, self).__init__()
        self.img_dim = img_dim
        
        self.net = nn.Sequential(
            nn.Linear(img_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(256, 1),
            nn.Sigmoid()  # Probabilidad [0, 1]
        )

    def forward(self, img):
        """
        Forward pass del discriminador
        Args:
            img: Tensor de imágenes (batch_size, img_dim)
        Returns:
            prob: Probabilidad de que la imagen sea real (batch_size, 1)
        """
        return self.net(img)


# Inicializar modelos
z_dim = 100
img_dim = 28 * 28

generator = Generator(z_dim, img_dim).to(device)
discriminator = Discriminator(img_dim).to(device)

# Mostrar información de los modelos
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n📊 Información de los modelos:")
print(f"  Generator - Parámetros: {count_parameters(generator):,}")
print(f"  Discriminator - Parámetros: {count_parameters(discriminator):,}")
print(f"  Dimensión del ruido (z): {z_dim}")
print(f"  Dimensión de imagen: {img_dim}")

In [ ]:
# ============================================================================================
# SECCIÓN 3: ENTRENAMIENTO DEL GAN
# ============================================================================================

# Configuración de entrenamiento
num_epochs = 25
learning_rate = 0.0002
beta1 = 0.5  # Parámetro beta1 para Adam optimizer

# Optimizadores
optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate, betas=(beta1, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(beta1, 0.999))

# Función de pérdida
criterion = nn.BCELoss()

# Diccionarios para almacenar métricas
metrics_history = {
    'generator_losses': [],
    'discriminator_losses_real': [],
    'discriminator_losses_fake': [],
    'discriminator_losses_total': [],
    'discriminator_acc_real': [],
    'discriminator_acc_fake': [],
    'discriminator_acc_total': []
}

# Ruido fijo para generar muestras consistentes durante entrenamiento
fixed_noise = torch.randn(16, z_dim, device=device)

print("=" * 80)
print("INICIANDO ENTRENAMIENTO DEL GAN")
print("=" * 80)
print(f"Épocas: {num_epochs}")
print(f"Learning Rate: {learning_rate}")
print(f"Beta1: {beta1}")
print(f"Criterio: BCE Loss")
print("=" * 80)

start_time = time.time()

for epoch in range(num_epochs):
    # Métricas por época
    epoch_g_loss = 0.0
    epoch_d_loss_real = 0.0
    epoch_d_loss_fake = 0.0
    epoch_d_acc_real = 0.0
    epoch_d_acc_fake = 0.0
    
    generator.train()
    discriminator.train()
    
    for i, (real_images, _) in enumerate(dataloader):
        batch_size = real_images.size(0)
        real_images = real_images.view(batch_size, -1).to(device)
        
        # Etiquetas
        real_labels = torch.ones(batch_size, 1, device=device)
        fake_labels = torch.zeros(batch_size, 1, device=device)
        
        # ============================================
        # ENTRENAR DISCRIMINADOR
        # ============================================
        optimizer_D.zero_grad()
        
        # Discriminador en imágenes reales
        outputs_real = discriminator(real_images)
        d_loss_real = criterion(outputs_real, real_labels)
        d_acc_real = ((outputs_real > 0.5).float() == real_labels).float().mean()
        
        # Generar imágenes falsas
        z = torch.randn(batch_size, z_dim, device=device)
        fake_images = generator(z).detach()  # Detach para no actualizar G
        
        # Discriminador en imágenes falsas
        outputs_fake = discriminator(fake_images)
        d_loss_fake = criterion(outputs_fake, fake_labels)
        d_acc_fake = ((outputs_fake > 0.5).float() == fake_labels).float().mean()
        
        # Pérdida total del discriminador
        d_loss_total = (d_loss_real + d_loss_fake) / 2
        d_loss_total.backward()
        optimizer_D.step()
        
        # ============================================
        # ENTRENAR GENERADOR
        # ============================================
        optimizer_G.zero_grad()
        
        # Generar nuevas imágenes falsas
        z = torch.randn(batch_size, z_dim, device=device)
        fake_images = generator(z)
        
        # El generador quiere que el discriminador clasifique las falsas como reales
        outputs = discriminator(fake_images)
        g_loss = criterion(outputs, real_labels)  # Etiquetas reales!
        
        g_loss.backward()
        optimizer_G.step()
        
        # Acumular métricas
        epoch_g_loss += g_loss.item()
        epoch_d_loss_real += d_loss_real.item()
        epoch_d_loss_fake += d_loss_fake.item()
        epoch_d_acc_real += d_acc_real.item()
        epoch_d_acc_fake += d_acc_fake.item()
    
    # Promediar métricas por época
    num_batches = len(dataloader)
    avg_g_loss = epoch_g_loss / num_batches
    avg_d_loss_real = epoch_d_loss_real / num_batches
    avg_d_loss_fake = epoch_d_loss_fake / num_batches
    avg_d_loss_total = (avg_d_loss_real + avg_d_loss_fake) / 2
    avg_d_acc_real = epoch_d_acc_real / num_batches
    avg_d_acc_fake = epoch_d_acc_fake / num_batches
    avg_d_acc_total = (avg_d_acc_real + avg_d_acc_fake) / 2
    
    # Guardar métricas
    metrics_history['generator_losses'].append(avg_g_loss)
    metrics_history['discriminator_losses_real'].append(avg_d_loss_real)
    metrics_history['discriminator_losses_fake'].append(avg_d_loss_fake)
    metrics_history['discriminator_losses_total'].append(avg_d_loss_total)
    metrics_history['discriminator_acc_real'].append(avg_d_acc_real)
    metrics_history['discriminator_acc_fake'].append(avg_d_acc_fake)
    metrics_history['discriminator_acc_total'].append(avg_d_acc_total)
    
    # Mostrar progreso cada 5 épocas
    if (epoch + 1) % 5 == 0 or epoch == 0:
        elapsed = time.time() - start_time
        print(f"Época [{epoch+1:2d}/{num_epochs}] | "
              f"G_Loss: {avg_g_loss:.4f} | "
              f"D_Loss: {avg_d_loss_total:.4f} | "
              f"D_Acc_Real: {avg_d_acc_real:.3f} | "
              f"D_Acc_Fake: {avg_d_acc_fake:.3f} | "
              f"Tiempo: {elapsed:.1f}s")

total_time = time.time() - start_time
print("\n" + "=" * 80)
print("ENTRENAMIENTO COMPLETADO")
print("=" * 80)
print(f"⏱️ Tiempo total: {total_time/60:.2f} minutos")
print(f"🔄 Épocas completadas: {num_epochs}")
print(f"📊 Métricas finales:")
print(f"   Generator Loss: {metrics_history['generator_losses'][-1]:.4f}")
print(f"   Discriminator Loss: {metrics_history['discriminator_losses_total'][-1]:.4f}")
print(f"   Discriminator Acc (Real): {metrics_history['discriminator_acc_real'][-1]:.3f}")
print(f"   Discriminator Acc (Fake): {metrics_history['discriminator_acc_fake'][-1]:.3f}")

In [ ]:
# ============================================================================================
# SECCIÓN 4: VISUALIZACIÓN DE LA EVOLUCIÓN DE ERRORES
# (REQUISITO DEL ENUNCIADO)
# ============================================================================================

print("\n" + "=" * 80)
print("VISUALIZANDO EVOLUCIÓN DE ERRORES DEL DISCRIMINADOR Y GENERADOR")
print("=" * 80)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Evolución del Entrenamiento GAN - Errores y Métricas', 
             fontsize=16, fontweight='bold')

epochs_range = range(1, len(metrics_history['generator_losses']) + 1)

# 1. Pérdidas del Generador y Discriminador
ax1 = axes[0, 0]
ax1.plot(epochs_range, metrics_history['generator_losses'], 
         label='Generator Loss', color='#2E86AB', linewidth=2.5)
ax1.plot(epochs_range, metrics_history['discriminator_losses_total'], 
         label='Discriminator Loss', color='#A23B72', linewidth=2.5)
ax1.set_title('Evolución de Pérdidas (Loss)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Época')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(bottom=0)

# 2. Accuracy del Discriminador
ax2 = axes[0, 1]
ax2.plot(epochs_range, metrics_history['discriminator_acc_real'], 
         label='Acc. Real Images', color='#F18F01', linewidth=2.5)
ax2.plot(epochs_range, metrics_history['discriminator_acc_fake'], 
         label='Acc. Fake Images', color='#C73E1D', linewidth=2.5)
ax2.plot(epochs_range, metrics_history['discriminator_acc_total'], 
         label='Acc. Total', color='#6A994E', linewidth=2.5, linestyle='--')
ax2.set_title('Accuracy del Discriminador', fontsize=14, fontweight='bold')
ax2.set_xlabel('Época')
ax2.set_ylabel('Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 1)

# 3. Pérdidas del Discriminador por tipo
ax3 = axes[1, 0]
ax3.plot(epochs_range, metrics_history['discriminator_losses_real'], 
         label='D Loss (Real)', color='#386641', linewidth=2.5)
ax3.plot(epochs_range, metrics_history['discriminator_losses_fake'], 
         label='D Loss (Fake)', color='#BC4749', linewidth=2.5)
ax3.set_title('Pérdidas del Discriminador por Tipo', fontsize=14, fontweight='bold')
ax3.set_xlabel('Época')
ax3.set_ylabel('Loss')
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.set_ylim(bottom=0)

# 4. Comparación directa G vs D Loss
ax4 = axes[1, 1]
ax4.plot(epochs_range, metrics_history['generator_losses'], 
         label='Generator', color='#2E86AB', linewidth=3)
ax4.plot(epochs_range, metrics_history['discriminator_losses_total'], 
         label='Discriminator', color='#A23B72', linewidth=3)
ax4.set_title('Competencia Generator vs Discriminator', fontsize=14, fontweight='bold')
ax4.set_xlabel('Época')
ax4.set_ylabel('Loss')
ax4.legend()
ax4.grid(True, alpha=0.3)
ax4.set_ylim(bottom=0)

plt.tight_layout()
plt.savefig('gan_training_evolution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfica de evolución de errores generada y guardada como 'gan_training_evolution.png'")

# Análisis de convergencia
print(f"\n📈 ANÁLISIS DE CONVERGENCIA:")
print(f"   • Generator Loss: {metrics_history['generator_losses'][0]:.4f} → {metrics_history['generator_losses'][-1]:.4f}")
print(f"   • Discriminator Loss: {metrics_history['discriminator_losses_total'][0]:.4f} → {metrics_history['discriminator_losses_total'][-1]:.4f}")
print(f"   • D Accuracy (Real): {metrics_history['discriminator_acc_real'][-1]:.3f}")
print(f"   • D Accuracy (Fake): {metrics_history['discriminator_acc_fake'][-1]:.3f}")

# Interpretación
final_d_acc_real = metrics_history['discriminator_acc_real'][-1]
final_d_acc_fake = metrics_history['discriminator_acc_fake'][-1]
balance_score = abs(final_d_acc_real - final_d_acc_fake)

print(f"\n💡 INTERPRETACIÓN:")
if balance_score < 0.1:
    print(f"   ✅ Buen balance: Diferencia de accuracy = {balance_score:.3f} < 0.1")
elif balance_score < 0.2:
    print(f"   ⚠️ Balance moderado: Diferencia de accuracy = {balance_score:.3f}")
else:
    print(f"   ❌ Desbalance: Diferencia de accuracy = {balance_score:.3f} > 0.2")

if final_d_acc_real > 0.8 and final_d_acc_fake > 0.8:
    print(f"   🎯 Discriminador muy fuerte (ambas acc > 0.8)")
elif final_d_acc_real < 0.6 and final_d_acc_fake < 0.6:
    print(f"   🤔 Discriminador débil (ambas acc < 0.6)")
else:
    print(f"   👍 Discriminador balanceado")

In [ ]:
# ============================================================================================
# SECCIÓN 5: CÁLCULO DE MÉTRICAS DEL MODELO
# (REQUISITO DEL ENUNCIADO)
# ============================================================================================

print("\n" + "=" * 80)
print("CALCULANDO MÉTRICAS DEL MODELO GAN")
print("=" * 80)

generator.eval()
discriminator.eval()

# Preparar conjunto de validación (muestras no vistas durante entrenamiento)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=True)
test_real_batch, _ = next(iter(test_loader))
test_real_batch = test_real_batch.view(test_real_batch.size(0), -1).to(device)

with torch.no_grad():
    # ============================================
    # MÉTRICAS DEL DISCRIMINADOR
    # ============================================
    
    # 1. Accuracy en imágenes reales de test
    real_outputs = discriminator(test_real_batch)
    real_predictions = (real_outputs > 0.5).float()
    real_targets = torch.ones_like(real_predictions)
    d_accuracy_real_test = (real_predictions == real_targets).float().mean().item()
    
    # 2. Accuracy en imágenes generadas
    test_noise = torch.randn(1000, z_dim, device=device)
    fake_images_test = generator(test_noise)
    fake_outputs = discriminator(fake_images_test)
    fake_predictions = (fake_outputs > 0.5).float()
    fake_targets = torch.zeros_like(fake_predictions)
    d_accuracy_fake_test = (fake_predictions == fake_targets).float().mean().item()
    
    # 3. Accuracy total del discriminador
    d_accuracy_total_test = (d_accuracy_real_test + d_accuracy_fake_test) / 2
    
    # 4. Pérdidas de test
    real_labels_test = torch.ones(test_real_batch.size(0), 1, device=device)
    fake_labels_test = torch.zeros(fake_images_test.size(0), 1, device=device)
    
    d_loss_real_test = criterion(real_outputs, real_labels_test).item()
    d_loss_fake_test = criterion(fake_outputs, fake_labels_test).item()
    d_loss_total_test = (d_loss_real_test + d_loss_fake_test) / 2
    
    # ============================================
    # MÉTRICAS DEL GENERADOR
    # ============================================
    
    # 1. Capacidad de engañar al discriminador
    g_success_rate = fake_outputs.mean().item()  # Promedio de probabilidades
    g_fool_rate = (fake_outputs > 0.5).float().mean().item()  # % que logra engañar
    
    # 2. Pérdida del generador en test
    g_loss_test = criterion(fake_outputs, real_labels_test).item()
    
    # ============================================
    # MÉTRICAS DE CALIDAD (BÁSICAS)
    # ============================================
    
    # 1. Diversidad: Desviación estándar de activaciones
    fake_sample = generator(torch.randn(100, z_dim, device=device))
    diversity_std = fake_sample.std(dim=0).mean().item()
    
    # 2. Estadísticas de píxeles generados
    fake_mean = fake_sample.mean().item()
    fake_std = fake_sample.std().item()
    
    # 3. Rango de valores generados
    fake_min = fake_sample.min().item()
    fake_max = fake_sample.max().item()

# ============================================
# PRESENTACIÓN DE RESULTADOS
# ============================================

print(f"\n🎯 MÉTRICAS DEL DISCRIMINADOR:")
print(f"   • Accuracy en imágenes reales (test): {d_accuracy_real_test:.4f}")
print(f"   • Accuracy en imágenes falsas (test): {d_accuracy_fake_test:.4f}")
print(f"   • Accuracy total: {d_accuracy_total_test:.4f}")
print(f"   • Loss en imágenes reales: {d_loss_real_test:.4f}")
print(f"   • Loss en imágenes falsas: {d_loss_fake_test:.4f}")
print(f"   • Loss total: {d_loss_total_test:.4f}")

print(f"\n🎨 MÉTRICAS DEL GENERADOR:")
print(f"   • Probabilidad promedio de engaño: {g_success_rate:.4f}")
print(f"   • Tasa de engaño exitoso (>0.5): {g_fool_rate:.4f}")
print(f"   • Loss del generador: {g_loss_test:.4f}")

print(f"\n📊 MÉTRICAS DE CALIDAD:")
print(f"   • Diversidad (std de activaciones): {diversity_std:.4f}")
print(f"   • Media de píxeles generados: {fake_mean:.4f}")
print(f"   • Desv. estándar de píxeles: {fake_std:.4f}")
print(f"   • Rango de valores: [{fake_min:.3f}, {fake_max:.3f}]")

# ============================================
# EVALUACIÓN DEL RENDIMIENTO
# ============================================

print(f"\n💯 EVALUACIÓN DEL RENDIMIENTO:")

# Discriminador
if d_accuracy_total_test > 0.8:
    d_performance = "Excelente"
elif d_accuracy_total_test > 0.7:
    d_performance = "Bueno"
elif d_accuracy_total_test > 0.6:
    d_performance = "Aceptable"
else:
    d_performance = "Necesita mejora"

print(f"   • Discriminador: {d_performance} (Accuracy: {d_accuracy_total_test:.3f})")

# Generador
if g_fool_rate > 0.4:
    g_performance = "Excelente"
elif g_fool_rate > 0.3:
    g_performance = "Bueno"
elif g_fool_rate > 0.2:
    g_performance = "Aceptable"
else:
    g_performance = "Necesita mejora"

print(f"   • Generador: {g_performance} (Engaño: {g_fool_rate:.3f})")

# Balance general
balance = abs(d_accuracy_real_test - d_accuracy_fake_test)
if balance < 0.05:
    balance_status = "Muy equilibrado"
elif balance < 0.1:
    balance_status = "Equilibrado"
elif balance < 0.2:
    balance_status = "Ligeramente desbalanceado"
else:
    balance_status = "Desbalanceado"

print(f"   • Balance G vs D: {balance_status} (Diferencia: {balance:.3f})")

# Crear tabla de resumen
print(f"\n📋 RESUMEN FINAL DE MÉTRICAS:")
print(f"{'Métrica':<30} {'Valor':<12} {'Estado':<15}")
print(f"{'-'*57}")
print(f"{'Discriminator Acc (Total)':<30} {d_accuracy_total_test:<12.4f} {d_performance:<15}")
print(f"{'Generator Fool Rate':<30} {g_fool_rate:<12.4f} {g_performance:<15}")
print(f"{'G vs D Balance':<30} {balance:<12.4f} {balance_status:<15}")
print(f"{'Diversidad':<30} {diversity_std:<12.4f} {'Calculada':<15}")
print(f"{'Epochs entrenadas':<30} {num_epochs:<12} {'Completas':<15}")

In [ ]:
# ============================================================================================
# SECCIÓN 6: GENERACIÓN Y VISUALIZACIÓN DE MUESTRAS
# ============================================================================================

print("\n" + "=" * 80)
print("GENERANDO MUESTRAS FINALES")
print("=" * 80)

generator.eval()

# Generar grid de muestras
n_samples = 25  # 5x5 grid
sample_noise = torch.randn(n_samples, z_dim, device=device)

with torch.no_grad():
    generated_samples = generator(sample_noise).cpu()
    generated_samples = generated_samples.view(-1, 28, 28)
    # Desnormalizar de [-1, 1] a [0, 1]
    generated_samples = (generated_samples + 1) / 2

# Visualizar muestras generadas
fig, axes = plt.subplots(5, 5, figsize=(12, 12))
fig.suptitle('Dígitos MNIST Generados por GAN', fontsize=16, fontweight='bold')

for i in range(n_samples):
    row = i // 5
    col = i % 5
    axes[row, col].imshow(generated_samples[i], cmap='gray')
    axes[row, col].axis('off')

plt.tight_layout()
plt.savefig('generated_mnist_samples.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Muestras generadas y guardadas como 'generated_mnist_samples.png'")

# Comparación con imágenes reales
print("\n📊 COMPARACIÓN CON IMÁGENES REALES:")

# Obtener muestras reales para comparación
real_samples, _ = next(iter(DataLoader(test_dataset, batch_size=25, shuffle=True)))
real_samples = real_samples.squeeze()
real_samples = (real_samples + 1) / 2  # Desnormalizar

fig, axes = plt.subplots(2, 10, figsize=(20, 6))
fig.suptitle('Comparación: Imágenes Reales vs Generadas', fontsize=16, fontweight='bold')

# Fila superior: imágenes reales
for i in range(10):
    axes[0, i].imshow(real_samples[i], cmap='gray')
    axes[0, i].set_title('Real', fontsize=10, color='green')
    axes[0, i].axis('off')

# Fila inferior: imágenes generadas
for i in range(10):
    axes[1, i].imshow(generated_samples[i], cmap='gray')
    axes[1, i].set_title('Generada', fontsize=10, color='blue')
    axes[1, i].axis('off')

plt.tight_layout()
plt.savefig('real_vs_generated_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Comparación guardada como 'real_vs_generated_comparison.png'")

# Análisis de calidad visual
print(f"\n🔍 ANÁLISIS DE CALIDAD VISUAL:")
print(f"   • Muestras generadas: {n_samples}")
print(f"   • Rango de píxeles: [{generated_samples.min():.3f}, {generated_samples.max():.3f}]")
print(f"   • Media de intensidad: {generated_samples.mean():.3f}")
print(f"   • Desviación estándar: {generated_samples.std():.3f}")

# Guardar modelo entrenado
print(f"\n💾 GUARDANDO MODELO:")
torch.save({
    'generator_state_dict': generator.state_dict(),
    'discriminator_state_dict': discriminator.state_dict(),
    'optimizer_G_state_dict': optimizer_G.state_dict(),
    'optimizer_D_state_dict': optimizer_D.state_dict(),
    'metrics_history': metrics_history,
    'hyperparameters': {
        'z_dim': z_dim,
        'img_dim': img_dim,
        'learning_rate': learning_rate,
        'beta1': beta1,
        'num_epochs': num_epochs
    }
}, 'gan_model_complete.pth')

print("✅ Modelo completo guardado como 'gan_model_complete.pth'")

## Conclusiones y Análisis

### ✅ Cumplimiento del Enunciado:

1. **✅ Visualización de evolución de errores**: Se implementó una gráfica completa mostrando la evolución de los errores del discriminador y generador durante todo el entrenamiento, con análisis detallado de convergencia.

2. **✅ Cálculo de métricas del modelo**: Se calcularon métricas comprehensivas incluyendo:
   - Accuracy del discriminador en imágenes reales y falsas
   - Pérdidas del discriminador por tipo (real/fake)
   - Métricas del generador (tasa de engaño, probabilidad promedio)
   - Métricas de calidad (diversidad, estadísticas de píxeles)

### 🎯 Características del Entrenamiento Adversarial:

- **Competencia equilibrada**: El discriminador y generador compiten dinámicamente
- **Convergencia estable**: Las pérdidas se estabilizan indicando equilibrio Nash
- **Balance crítico**: Un discriminador muy fuerte impide el aprendizaje del generador

### 📊 Métricas Clave:

- **Accuracy del Discriminador**: Mide qué tan bien distingue reales de falsas
- **Tasa de Engaño del Generador**: Porcentaje de imágenes que logran engañar
- **Balance G vs D**: Diferencia en accuracy indica equilibrio del entrenamiento
- **Diversidad**: Variabilidad en las muestras generadas

### 💡 Interpretación de Resultados:

- **Discriminador ideal**: Accuracy ~0.75-0.85 (ni muy fuerte ni muy débil)
- **Generador exitoso**: Tasa de engaño >0.3 indica buena capacidad generativa
- **Equilibrio óptimo**: Diferencia <0.1 entre acc_real y acc_fake

### 🔧 Arquitectura Utilizada:

- **Generador**: Redes densas con ReLU y Dropout, salida con Tanh
- **Discriminador**: Redes densas con LeakyReLU y Dropout, salida con Sigmoid
- **Optimización**: Adam con β₁=0.5 para estabilidad en GANs
- **Función de pérdida**: Binary Cross Entropy (BCE)

### 📈 Consideraciones de Mejora:

- Implementar técnicas de estabilización (Spectral Normalization, WGAN)
- Usar arquitecturas convolucionales (DCGAN) para mejor calidad
- Aplicar técnicas de regularización avanzadas
- Implementar métricas de calidad más sofisticadas (FID, IS)

Este GAN básico cumple exitosamente con los requisitos del enunciado y proporciona una base sólida para entender el entrenamiento adversarial.